In [1]:
days = 20*5

DemandB2 = [1740 2400 2000 2500 1900 0]

Ffactor = 180;
Gfactor = 20;
C21factor = 30;
B21factor = 30;

InvB2 = zeros(days+50,1)
InvF = zeros(days+50,1)
InvG = zeros(days+50,1)
InvC21 = zeros(days+50,1)
InvB21 = zeros(days+50,1)

InvB21[1:end] = 5;

cycleB2 = 975


t=1;
demandMonth = 1
nOrders=0
noOrder = true
while t < days
    if t >= (demandMonth*20)
        InvB2[t:end] -= DemandB2[demandMonth]
        demandMonth += 1
    end
    if InvB2[t] > DemandB2[demandMonth]
        keepProduction = false
    else
        keepProduction = true
    end
    Fleft = (DemandB2[demandMonth] - InvB2[t]) * Ffactor
    Gleft = (DemandB2[demandMonth] - InvB2[t]) * Gfactor
    B21left = (DemandB2[demandMonth] - InvB2[t]) * B21factor
    C21left = (DemandB2[demandMonth] - InvB2[t]) * C21factor
    if ((InvF[t] < Fleft) || (InvG[t] < Gleft) || (InvC21[t] < C21left) ||(InvB21[t] < B21left)) && noOrder && keepProduction
        nOrders+=1
        tLast = t
        tAdd = round(Int64,2*randn(1)[1] + 5)
        if tAdd < 0
            t+=0
        else
            t+=tAdd
        end
        
        InvF[tLast:t] = InvF[tLast]
        InvG[tLast:t] = InvG[tLast]
        InvB21[tLast:t] = InvB21[tLast]
        InvC21[tLast:t] = InvC21[tLast]
        InvB2[tLast:t] = InvB2[tLast]
        
        InvF[t:end] += max(DemandB2[demandMonth]* Ffactor, cycleB2 * Ffactor * 7)
        InvG[t:end] += max(DemandB2[demandMonth]* Gfactor, cycleB2 * Gfactor * 7)
        InvB21[t:end] += max(DemandB2[demandMonth]* B21factor, cycleB2 * B21factor * 7)
        InvC21[t:end] += max(DemandB2[demandMonth]* C21factor, cycleB2 * C21factor * 7)
    else
        if InvB2[t] > DemandB2[demandMonth]
            keepProduction = false
        else
            keepProduction = true
        end
        while ((InvF[t]-Ffactor)>0) && ((InvC21[t]-C21factor)>0) && ((InvG[t]-Gfactor)>0) && ((InvB21[t]-B21factor)>0) && ((t+1)<= days) && keepProduction          
            
            prod = round(Int64, 200*randn(1)[1] + cycleB2/2)
            nF = div(InvF[t], Ffactor)
            nG = div(InvG[t], Gfactor)
            nB21 = div(InvB21[t], B21factor)
            nC21 = div(InvC21[t], C21factor)
            
            nB2 = min(nF, nG, nB21, nC21, prod)
            
            InvB2[t:end] += nB2
            InvF[t:end] -= nB2*Ffactor
            InvG[t:end] -= nB2*Gfactor
            InvB21[t:end] -= nB2*B21factor    
            InvC21[t:end] -= nB2*C21factor 
            defect = rand()/10
            
            InvB2[t:end] -= round(Int64, InvB2[t+1]*defect)
            if InvB2[t] > DemandB2[demandMonth]
                break
            end
            t+=1
            if t >= (demandMonth*20)
                InvB2[t:end] -= DemandB2[demandMonth]
                demandMonth += 1
            end
            Fleft = (DemandB2[demandMonth] - InvB2[t]) * Ffactor
            Gleft = (DemandB2[demandMonth] - InvB2[t]) * Gfactor
            B21left = (DemandB2[demandMonth] - InvB2[t]) * B21factor
            C21left = (DemandB2[demandMonth] - InvB2[t]) * C21factor
            if ((InvF[t] < Fleft) || (InvG[t] < Gleft) || (InvB21[t] < B21left)|| (InvC21[t] < C21left)) && noOrder
                noOrder = false
                nOrders += 1
                tLast = t
                tAdd = round(Int64,2*randn(1)[1] + 5)
                if tAdd < 0
                    tOrder+=0
                else
                    tOrder+=tAdd
                end

                InvF[tLast:tOrder] = InvF[tLast]
                InvG[tLast:tOrder] = InvG[tLast]
                InvB21[tLast:tOrder] = InvB21[tLast]
                InvC21[tLast:tOrder] = InvC21[tLast]
                InvB2[tLast:tOrder] = InvB2[tLast]


            end
            if !noOrder
                if !(((InvF[t]-Ffactor)>0) && ((InvC21[t]-C21factor)>0) && ((InvG[t]-Gfactor)>0) && ((InvB21[t]-B21factor)>0) && ((t+1)<= days) && keepProduction)
                    noOrder = true
                    t = tOrder
                    InvF[tOrder:end] += max(Fleft* Ffactor, cycleB2 * Ffactor * 7)
                    InvG[tOrder:end] += max(Gleft* Gfactor, cycleB2 * Gfactor * 7)
                    InvB21[tOrder:end] += max(B21left* B21factor, cycleB2 * B21factor * 7)
                    InvC21[tOrder:end] += max(C21left* C21factor, cycleB2 * C21factor * 7)
                end
            end
            
        end
    end
    t+=1
end




In [4]:
using Plots
plot(1:(days+50), InvF/Ffactor, label="F scaled", title="Inventory over 5 months")
plot!(1:(days+50), InvG/Gfactor, label = "G scaled")
plot!(1:(days+50), InvC21/C21factor, label="C21 scaled")
plot!(1:(days+50), InvB21/B21factor, label="B21 scaled")
plot!(1:(days+50), InvB2, label="B2 actual")
using Images
using FileIO
savefig("b2invall.png")

In [5]:
plot(1:(days+50), InvB2, label="B2", title="Inventory of B2")
using Images
using FileIO
savefig("b2inv.png")